In [5]:
import pandas as pd
import pyodbc
import os

In [4]:
# Read Excel file into a DataFrame
excel_file = os.getcwd() + '/github/Workana/text_to_sql/database/pnad_Trimestral.xlsx'
df = pd.read_excel(excel_file)
df = df.fillna(0)
# SQL Server connection parameters
server = 'host.docker.internal'
database = 'llm_data'
username = 'sa'
password = 'Zantut123@'
driver = '{ODBC Driver 18 for SQL Server}'

# Connect to the SQL Server database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
conn = pyodbc.connect(conn_str)

# Define table name
table_name = 'PNAD_TRIMESTRAL'

# Create SQL query to create table
create_table_query = f'''
CREATE TABLE {table_name} (
    {', '.join([f'{column} VARCHAR(MAX)' for column in df.columns])}
)
'''

# Execute the create table query
cursor = conn.cursor()
cursor.execute(create_table_query)
conn.commit()

# Insert data from DataFrame into the table
for index, row in df.iterrows():
    insert_query = f'''
    INSERT INTO {table_name} ({', '.join(df.columns)})
    VALUES ({', '.join([f"?"] * len(df.columns))})
    '''
    cursor.execute(insert_query, tuple(row))
    conn.commit()

# Close connection
conn.close()

print(f'Table "{table_name}" created successfully and data inserted from Excel file.')


Table "PNAD_TRIMESTRAL" created successfully and data inserted from Excel file.
